## Project : To scrape cutomer reviews of british airlines and analyse the data collected for usable insights

#### Internship project for FORAGE British Airways Data Science project 

### Task 1
### Web scraping and analysis

Scraping data from Skytrax
If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use Python and BeautifulSoup to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [ ]:
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from textblob import TextBlob
from wordcloud import WordCloud
from string import digits
import requests
import pandas as pd
import nltk
import string
import seaborn as sns
import re
import numpy as np
nltk.download("stopwords")
stop_words = set(stopwords.words('english'))

In [ ]:
#importing all the required libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud

In [ ]:
# #scraping data fron skytrax
# base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
# pages = 36
# page_size = 100

# reviews = []
# stars = []
# date = []
# country = []

# # for i in range(1, pages + 1):
# for i in range(1, pages + 1):

#     print(f"Scraping page {i}")

#     # Create URL to collect links from paginated data
#     url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

#     # Collect HTML data from this page
#     response = requests.get(url)

#     # Parse content
#     content = response.content
#     parsed_content = BeautifulSoup(content, 'html.parser')
#     for para in parsed_content.find_all("div", {"class": "text_content"}):
#         reviews.append(para.get_text())

#     for para in parsed_content.find_all("time"):
#         date.append(para.get_text())

In [ ]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 10
page_size = 100

reviews = []

# for i in range(1, pages + 1):
for i in range(1, pages + 1):

    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

Saving the reviews in a data frame

In [ ]:
data = {"reviews":reviews}
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose()
df.head()

In [ ]:
# downloading file
df.to_csv("BA_reviews.csv")

#### TASK 2
#### Data cleaning

In [ ]:
import pandas as pd

csv_path = "BA_reviews.csv"
ds = pd.read_csv(csv_path)
ds.reset_index(drop=True, inplace=True)

print(ds['reviews'])

In [ ]:
ds.info()
ds.describe()

There are no null values/ missing values in the dataset.
The dataset contains 1000 unique entries.
Removing (✅ Trip Verified | and Not Verified | ) to clean the data.
Removing any leading or trailing spaces.
Turning the review string to all lower case.


In [ ]:
ds['reviews'] = ds['reviews'].str.strip()
ds

In [ ]:
ds.reviews= ds.reviews.str.replace('.*\|', '', regex=True)

In [ ]:
ds

In [ ]:
ds['reviews']= ds['reviews'].str.lower()
print(ds)

## TASK 3
#### Sentiment analysis of reviews using nltk 

Remove punctutaions

In [ ]:
ds_new = ds.copy()

In [ ]:
ds_new

In [ ]:
#remove punctuation
ds_new['reviews'] = ds_new['reviews'].str.replace('[^\w\s]','')
print(ds_new['reviews'])

Tokenize the review column

In [ ]:
ds_new

In [ ]:
# tokenize
print(ds_new.iloc[1,1])
ds_new['reviews'] = ds_new.apply(lambda row: nltk.word_tokenize(row['reviews']), axis=1)
print(ds_new.iloc[1,1])

Removing stopwords

In [ ]:
ds_tksd = ds_new.copy()

In [ ]:
ds_tksd['reviews'] = ds_tksd['reviews'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))
print(ds_tksd.head(20))

Calculate polarity to gather sentiment tags.

In [ ]:
def polarity_calc(text):
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return None
    

    
def tag_cal(num):
    if num<0:
        return 'Negative'
    elif num>0:
        return 'Positive'
    else:
        return 'Neutral'
        
    
ds_tksd['polarity'] = ds_tksd['reviews'].apply(polarity_calc)


ds_tksd['tag'] = ds_tksd['polarity'].apply(tag_cal)


print(ds_tksd)

## TASK 4 
#### Analyze 

The percentage of various types of tags.


In [ ]:
(ds_tksd.groupby('tag').size()/ds_tksd['tag'].count())*100

Number of each kind of reviews

## TASK 5
#### Visualizing the results

In [ ]:
text = " "
for ind in ds_tksd.index:
    if ds_tksd['tag'][ind] == "Positive":
        text = text + ds_tksd['reviews'][ind]
      
wordcloud_positive = WordCloud().generate(text)


# Display the generated image:
plt.imshow(wordcloud_positive, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
text2= " "        
for ind in ds_tksd.index:
    if ds_tksd['tag'][ind] == "Negative":
        text2 = text2 + ds_tksd['reviews'][ind]  
wordcloud_negative = WordCloud().generate(text2)
plt.imshow(wordcloud_negative, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# ds_tksd['tag'].value_counts().plot(kind='bar')
sns.set(font_scale = 1.4)
ds_tksd['tag'].value_counts().plot(kind='bar', figsize=(7, 6), rot=0)
plt.xlabel("Sentiment", labelpad=14)
plt.ylabel("No of reviews", labelpad=14)
plt.title("Sentient counts", y=1.02);

In [ ]:
plt.pie(ds_tksd['tag'].value_counts(), labels = ds_tksd['tag'].value_counts().index, explode = (0, 0, 0.25), autopct='%1.1f%%', shadow=False)

Let's compare this analysis with vader analysis based on the lemmatized text

In [ ]:
"""This punkt tokenizer divides a text into a list of sentences by using an unsupervised algorithm to build a model for abbreviation words, 
collocations, and words that start sentences. """

nltk.download('punkt')
from nltk import pos_tag
nltk.download('wordnet')
from nltk.corpus import wordnet

In [ ]:
ds_final = ds.copy()
ds_final['reviews'] = ds_final['reviews'].str.replace('[^\w\s]','')
ds_final

In [ ]:
ds_new

In [ ]:
import re

# Define a function to clean the text
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', str(text))
    return text

# Cleaning the text in the review column
ds_final['Cleaned Reviews'] = ds_final['reviews'].apply(clean)
ds_final.head()

In [ ]:
#The nltk.corpus package defines a collection of corpus reader classes, which can be used to access the contents of a diverse set of corpora.


nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

# POS tagger dictionary
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    #print(tags)
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
          newlist.append(tuple([word, pos_dict.get(tag[0])]))
          #print(tag[0])
          #print(pos_dict.get(tag[0]))
    return newlist 

ds_final['POS tagged'] = ds_final['Cleaned Reviews'].apply(token_stop_pos)
ds_final.head()

In [ ]:
# Obtaining the stem words – Lemmatization

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
     if not pos:
        lemma = word
        lemma_rew = lemma_rew + " " + lemma
     else:
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

ds_final['Lemma'] = ds_final['POS tagged'].apply(lemmatize)
ds_final.head()

In [ ]:
ds_final[['reviews','Lemma']]

# Sentiment Analysis using VADER

VADER stands for Valence Aware Dictionary and Sentiment Reasoner.

Vader sentiment not only tells if the statement is positive or negative along with the intensity of emotion.



In [ ]:
!pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()


# function to calculate vader sentiment
def vadersentimentanalysis(review):
    vs = analyzer.polarity_scores(review)
    return vs['compound']

ds_final['Sentiment'] = ds_final['Lemma'].apply(vadersentimentanalysis)

# function to analyse
def vader_analysis(compound):
    if compound >= 0.5:
        return 'Positive'
    elif compound < 0 :
        return 'Negative'
    else:
        return 'Neutral'
ds_final['Analysis'] = ds_final['Sentiment'].apply(vader_analysis)
ds_final.head()

In [ ]:
vader_counts = ds_final['Analysis'].value_counts()
vader_counts

# Visual Representation

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(15,7))

plt.subplot(1,3,2)
plt.title("Reviews Analysis")
plt.pie(vader_counts.values, labels = vader_counts.index, explode = (0, 0, 0.25), autopct='%1.1f%%', shadow=False)

# Wordcloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=100,
        max_font_size=30,
        scale=3,
        random_state=1)

    wordcloud=wordcloud.generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(ds_final.Lemma)

In [ ]:
data = df.copy()

In [ ]:
data

In [ ]:
data['verified?'] = data['reviews'].apply(lambda x: x.split('|')[0])
data['reviews'] = data['reviews'].str.replace('.*\|', '', regex=True)

In [ ]:
data

In [ ]:
data_test = data[data['verified?'].str.contains('Verified')]

In [ ]:
data_test.loc[(data_test['verified?'] != '✅ Trip Verified ') & (data_test['verified?'] != None),'verified?'] = 'Not Verified'
data_test.loc[data_test['verified?'] == '✅ Trip Verified ','verified?'] = 'Trip Verified'
data_test

In [ ]:
data_test['verified?'].value_counts()

In [ ]:
data

In [ ]:
import spacy, gensim
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from pprint import pprint

# Plotting tools
import matplotlib.pyplot as plt

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['reviews'].tolist()))

print(data_words[0])

In [ ]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

In [ ]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [ ]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

In [ ]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=4,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model) 

In [ ]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [ ]:
df_document_topic.shape

In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

In [ ]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

In [ ]:
plt.bar(range(1,5,1),[25,573,169,233], color="blue")
plt.ylabel("Number of Reviews")
plt.title("Number of reviews per topic")
x_labels = (["Positive\nCustomer\nExperience", "In-flight\namenities\nand services",
            "Boarding\n& departure\nprocess","Booking\n& refund\nprocess"])
plt.xticks(range(1,5), x_labels)
# Show the graph
plt.show()

In [ ]:
vectorizer

In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

In [ ]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

In [ ]:
data['sentiment'] = data['reviews'].apply(lambda x: sentiment_pipeline(x, max_length=512, truncation=True)[0]["label"])
data['confidence'] = data['reviews'].apply(lambda x: sentiment_pipeline(x, max_length=512, truncation=True)[0]["score"])

In [ ]:
data['sentiment'].value_counts()

In [ ]:
data.groupby('verified?')['sentiment'].value_counts()

In [ ]:
data.groupby('verified?')['sentiment'].value_counts().unstack().plot.bar(stacked=True)

In [ ]:
data.head(5)

In [ ]:
topics_ = df_document_topic['dominant_topic'].tolist()

In [ ]:
data['topic'] = topics_

In [ ]:
data.head()

In [ ]:
data[data['verified?'] == '✅ Trip Verified '].groupby(['topic'])['sentiment'].value_counts().unstack().plot.bar(stacked=True)

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.util import ngrams
from nltk import FreqDist

In [ ]:
import string
from nltk.corpus import stopwords

# Convert to lower case
data['review_text'] = data['reviews'].str.lower()

# Remove punctuation and numbers
data['review_text'] = data['review_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation + string.digits)))

# Remove stop words
stop_words = stopwords.words("english")
stop_words.extend(['british','airway','airline','ba','flight'])
stop_words = set(stop_words)
data['review_text'] = data['review_text'].apply(lambda x: " ".join(word for word in x.split() if word.lower() not in stop_words))


# Perform lemmatization 
lemmatizer = WordNetLemmatizer()
data['review_text'] = data['review_text'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) 
                                                                    for word in x.split() if len(word)> 3]))

In [ ]:
# Get bigrams and their frequency
bigrams = list(ngrams(data[data['sentiment'] == 'POSITIVE']['review_text'].str.cat(sep=' ').split(), 2))
bigrams_fd = FreqDist(bigrams)

In [ ]:
bigrams_fd = {'_'.join(bigram): freq for bigram, freq in bigrams_fd.items()}

In [ ]:
wordcloud = WordCloud(width=800, height=400, min_font_size = 10, background_color ='white',)
# Generate the word cloud
wordcloud.generate_from_frequencies(bigrams_fd)

# Show the word cloud
plt.figure(figsize=(10,10), facecolor = None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
# Get bigrams and their frequency
bigrams_ng = list(ngrams(data[data['sentiment'] == 'NEGATIVE']['review_text'].str.cat(sep=' ').split(), 2))
bigrams_fd_ng = FreqDist(bigrams_ng)
bigrams_fd_ng = {'_'.join(bigram): freq for bigram, freq in bigrams_fd_ng.items()}

In [ ]:
igrams_fd_ng = FreqDist(bigrams_ng)
bigrams_fd_ng = {''.join(bigram): freq for bigram, freq in bigrams_fd_ng.items()}
wordcloud = WordCloud(width=800, height=400, min_font_size = 10, background_color ='white',)
# Generate the word cloud
wordcloud.generate_from_frequencies(bigrams_fd_ng)

# Show the word cloud
plt.figure(figsize=(10,10), facecolor = None)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()